In [1]:
import keras
import numpy as np
import math
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Model
from keras.layers import Input, Conv2D, GlobalAveragePooling2D, Dropout
from keras.layers import Activation, BatchNormalization, Add, Reshape, DepthwiseConv2D,ReLU

from keras import backend as K
from keras import optimizers, regularizers
from keras.callbacks import LearningRateScheduler, TensorBoard

Using TensorFlow backend.


## load the data

In [5]:
def color_preprocessing(x_train,x_test):
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    mean = [125.307, 122.95, 113.865]
    std  = [62.9932, 62.0887, 66.7048]
    for i in range(3):
        x_train[:,:,:,i] = (x_train[:,:,:,i] - mean[i]) / std[i]
        x_test[:,:,:,i] = (x_test[:,:,:,i] - mean[i]) / std[i]
    return x_train, x_test

# load data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test  = keras.utils.to_categorical(y_test, num_classes)
x_train, x_test = color_preprocessing(x_train, x_test)

   237568/170498071 [..............................] - ETA: 21:17:17

Exception: URL fetch failure on https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz : None -- retrieval incomplete: got only 245760 out of 170498071 bytes

## Module and Layer

In [3]:
def block(inputs, out_chans, k, s):

    x = Conv2D(out_chans, k, padding='same', strides=s)(inputs)
    x = BatchNormalization()(x)
    x = ReLU(max_value=6.0)(x)
    return x


def inverted_residual_block(inputs, out_chans, k, t, s, r=False):
    
    # Depth
    tchannel = K.int_shape(inputs)[-1] * t

    x = block(inputs, tchannel, 1, 1)
    # 注意这里depth_multiplier这个参数的含义。
    x = DepthwiseConv2D(k, strides=s, depth_multiplier=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU(max_value=6.0)(x)
    
    x = Conv2D(out_chans, (1, 1), strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    if r:
        x = Add()([x, inputs])

    return x


def make_layer(inputs, out_chans, k, t, s, n):

    x = inverted_residual_block(inputs, out_chans, k, t, s)

    for i in range(1, n):
        x = inverted_residual_block(x, out_chans, k, t, 1, True)

    return x

## Model

In [4]:
def MobileNetv2(input_shape, k):
    
    inputs = Input(shape=input_shape)

    x = block(inputs, 32, (3, 3), s=(2, 2))
    x = inverted_residual_block(x, 16, (3, 3), t=1, s=1)
    
    x = make_layer(x, 24, (3, 3), t=6, s=2, n=2)
    x = make_layer(x, 32, (3, 3), t=6, s=2, n=3)
    x = make_layer(x, 64, (3, 3), t=6, s=2, n=4)
    x = make_layer(x, 96, (3, 3), t=6, s=1, n=3)
    x = make_layer(x, 160, (3, 3), t=6, s=2, n=3)
    x = inverted_residual_block(x, 320, (3, 3), t=6, s=1)

    x = block(x, 1280, (1, 1), s=(1, 1))#[n,7,7,c]
    x = GlobalAveragePooling2D()(x)#[n,c]
    x = Reshape((1, 1, 1280))(x)#[n,1,1,1280]
    x = Dropout(0.3, name='Dropout')(x)
    x = Conv2D(k, (1, 1), padding='same')(x)#[n,1,1,k]

    x = Activation('softmax', name='softmax')(x)
    output = Reshape((k,))(x)

    model = Model(inputs, output)

    return model

In [20]:
def scheduler(epoch):
    if epoch < 150:
        return 0.1
    if epoch < 225:
        return 0.01
    return 0.001

# set optimizer
sgd = optimizers.SGD(lr=.1, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#scalar
# set callback
tb_cb = TensorBoard(log_dir=log_filepath, histogram_freq=0)
change_lr = LearningRateScheduler(scheduler)
cbks = [change_lr,tb_cb]

# set data augmentation
datagen = ImageDataGenerator(horizontal_flip=True,
                             width_shift_range=0.125,
                             height_shift_range=0.125,
                             fill_mode='constant',cval=0.)
datagen.fit(x_train)

# start training
model.fit_generator(datagen.flow(x_train, y_train,batch_size=batch_size), 
                    steps_per_epoch=iterations, 
                    epochs=epochs, 
                    callbacks=cbks,
                    validation_data=(x_test, y_test))
model.save('mobilenetv2.h5')
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_42 (Conv2D)              (None, 112, 112, 32) 896         input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_61 (BatchNo (None, 112, 112, 32) 128         conv2d_42[0][0]                  
__________________________________________________________________________________________________
re_lu_42 (ReLU)                 (None, 112, 112, 32) 0           batch_normalization_61[0][0]     
____________________________________________________________________________________________